In [17]:
import numpy as np
from sklearn.datasets import load_iris


In [49]:
iris = load_iris()
X = iris.data
y = iris.target

print(X.shape)  

(150, 4)


In [39]:
X = X[:100, :]
y = y[:100]

K =5


print(X.shape)
print(y.shape)  

(100, 4)
(100,)


In [32]:
folds = np.random.choice(K, size=X.shape[0], replace=True)
print(folds)

[1 4 3 0 1 4 0 3 0 4 4 2 4 0 0 1 3 4 1 2 2 2 3 0 2 3 1 2 0 0 0 2 3 1 3 2 0
 3 4 3 1 3 1 0 0 1 2 2 3 3 1 1 1 0 1 2 1 1 0 2 1 2 4 4 2 2 4 2 0 0 1 1 3 1
 4 3 1 0 3 2 4 4 0 3 1 3 4 1 0 1 3 1 2 4 1 4 2 2 0 4]


In [40]:
X_test = X[folds == 0]
y_test = y[folds == 0]

X_train = X[folds != 0]
y_train = y[folds != 0]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)   

(80, 4) (80,)
(20, 4) (20,)


In [41]:
import pandas as pd

In [42]:
prior_probs = pd.Series(y_train).value_counts(normalize=True)

In [43]:
def PDF(x, mean, var):
    first = 1 / np.sqrt(2 * np.pi * var)
    second = np.exp(- (x - mean) ** 2 / (2 * var))
    
    return first * second

In [44]:
mean_class0 = X_train[y_train == 0].mean(axis=0)
var_class0 = X_train[y_train == 0].var(axis=0)

mean_class1 = X_train[y_train == 1].mean(axis=0)
var_class1 = X_train[y_train == 1].var(axis=0)

print("mean_class0:", mean_class0)
print("var_class0:", var_class0)
print("mean_class1:", mean_class1)
print("var_class1:", var_class1)

mean_class0: [5.03157895 3.45263158 1.45789474 0.23684211]
var_class0: [0.1000554  0.15249307 0.02454294 0.00759003]
mean_class1: [5.9452381  2.7952381  4.28571429 1.32857143]
var_class1: [0.25771542 0.09473923 0.19360544 0.03870748]


In [ ]:
probs = []
y_pred = []

for i in range(X_test.shape[0]):
    # Calculate likelihoods for class 0
    likelihood_class0 = PDF(X_test[i, :], mean_class0, var_class0)
    likelihood_product_class0 = likelihood_class0.prod()
    
    # Calculate likelihoods for class 1
    likelihood_class1 = PDF(X_test[i, :], mean_class1, var_class1)
    likelihood_product_class1 = likelihood_class1.prod()
    
    # Calculate posteriors
    posterior_class0 = likelihood_product_class0 * prior_probs[0]
    posterior_class1 = likelihood_product_class1 * prior_probs[1]
    
    # Normalize
    denom = posterior_class0 + posterior_class1
    posterior_class0 /= denom
    posterior_class1 /= denom
    
    # Store results
    probs.append([posterior_class0, posterior_class1])
    y_pred.append(int(np.argmax([posterior_class0, posterior_class1])))

    print(f"Test sample {i}: Posterior Class 0 = {posterior_class0:.4f}, Posterior Class 1 = {posterior_class1:.4f}, Predicted Class = {y_pred[-1]}")

Test sample 0: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 1: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 2: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 3: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 4: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 5: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 6: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 7: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 8: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 9: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 0
Test sample 10: Posterior Class 0 = 1.0000, Posterior Class 1 = 0.0000, Predicted Class = 

In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
cm = confusion_matrix(y_test, y_pred)

tp = cm[0,0]
tn = cm[1,1]
fp = cm[1,0]
fn = cm[0,1]

accuaracy = (tp + tn) / cm.sum()
precision = tp / (tp + fp)
recall = tp / (tp + fn)

In [48]:
print(f"Accuracy: {accuaracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"\nConfusion Matrix:\n{cm}")
print(f"\nTrue Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000

Confusion Matrix:
[[12  0]
 [ 0  8]]

True Positives: 12
True Negatives: 8
False Positives: 0
False Negatives: 0
